In [1]:
from google.colab import drive
drive.mount('/content/drive')
import numpy as np
from matplotlib import pyplot as plt
import pandas as pd
from os import listdir
import librosa as lb

from IPython.display import HTML, display
import time
import IPython

def ls(ruta = '.'):
    return listdir(ruta)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


#Base de datos:

Elements contained in the filenames:

1.Patient number (101,102,...,226)

2.Recording index

3.Chest location (Trachea (Tc), {Anterior (A), Posterior (P),,Lateral (L)}{left (l), right (r)})

4.Acquisition mode (sequential/single channel (sc),simultaneous/multichannel (mc))

5.Recording equipment (AKG C417L Microphone, 3M Littmann Classic II SE Stethoscope, 3M Litmmann 3200 Electronic Stethoscope, WelchAllyn Meditron Master Elite Electronic Stethoscope)

The abbreviations used in the diagnosis file are:
- COPD: Chronic Obstructive Pulmonary Disease
- LRTI: Lower Respiratory Tract Infection
- URTI: Upper Respiratory Tract Infection

In [2]:
directory= "/content/drive/My Drive/Respiratory_Sound_Database/audio_and_txt_files/"#depende de cada drive
archivos=ls(directory)#lista de todos los archivos en la carpeta
archivos=[archivos[i][0:-4] for i in range(len(archivos)) if '.txt' in archivos[i]] #solo los nombres

In [3]:
matriz1={'File':[],
        'Patient':[],
         'Equipment':[],
         'mc/sc':[]
}
for archivo in archivos:
  matriz1['File'].append(archivo)
  matriz1['Patient'].append(int(archivo[0:3]))
  matriz1['Equipment'].append(archivo[14:])
  matriz1['mc/sc'].append(archivo[11:13])

In [4]:
df1=pd.DataFrame(matriz1,index=None)
df1

,File,Patient,Equipment,mc/sc
0,185_1b1_Pl_sc_Litt3200,185,Litt3200,sc
1,185_1b1_Lr_sc_Litt3200,185,Litt3200,sc
2,185_1b1_Pr_sc_Litt3200,185,Litt3200,sc
3,186_2b2_Lr_mc_AKGC417L,186,AKGC417L,mc
4,186_2b2_Al_mc_AKGC417L,186,AKGC417L,mc
...,...,...,...,...
915,183_1b1_Tc_sc_Meditron,183,Meditron,sc
916,184_1b1_Ar_sc_Meditron,184,Meditron,sc
917,185_1b1_Al_sc_Litt3200,185,Litt3200,sc
918,185_1b1_Ar_sc_Litt3200,185,Litt3200,sc


In [5]:
diagnosis=pd.read_csv("/content/drive/My Drive/Respiratory_Sound_Database/patient_diagnosis.csv",header=None,names=['paciente','diagnostico'])

In [6]:
diagnosticos=[]
for i in df1['Patient']:
  diagnosticos.append(diagnosis['diagnostico'][diagnosis['paciente']==i].values[0])

df1['Diagnosis']=diagnosticos

In [7]:
df1

,File,Patient,Equipment,mc/sc,Diagnosis
0,185_1b1_Pl_sc_Litt3200,185,Litt3200,sc,COPD
1,185_1b1_Lr_sc_Litt3200,185,Litt3200,sc,COPD
2,185_1b1_Pr_sc_Litt3200,185,Litt3200,sc,COPD
3,186_2b2_Lr_mc_AKGC417L,186,AKGC417L,mc,COPD
4,186_2b2_Al_mc_AKGC417L,186,AKGC417L,mc,COPD
...,...,...,...,...,...
915,183_1b1_Tc_sc_Meditron,183,Meditron,sc,Healthy
916,184_1b1_Ar_sc_Meditron,184,Meditron,sc,Healthy
917,185_1b1_Al_sc_Litt3200,185,Litt3200,sc,COPD
918,185_1b1_Ar_sc_Litt3200,185,Litt3200,sc,COPD


In [8]:
print(np.unique(df1['Equipment'].values,return_counts=True))
print(np.unique(df1['Diagnosis'].values,return_counts=True))

(array(['AKGC417L', 'Litt3200', 'LittC2SE', 'Meditron'], dtype=object), array([646,  60,  87, 127]))
(array(['Asthma', 'Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy',
       'LRTI', 'Pneumonia', 'URTI'], dtype=object), array([  1,  16,  13, 793,  35,   2,  37,  23]))


In [9]:
print(np.unique(diagnosis['diagnostico'].values,return_counts=True))

(array(['Asthma', 'Bronchiectasis', 'Bronchiolitis', 'COPD', 'Healthy',
       'LRTI', 'Pneumonia', 'URTI'], dtype=object), array([ 1,  7,  6, 64, 26,  2,  6, 14]))


# Archivos de audio

Cada archivo de audio viene acopañado de anotaciones de inicio, fin de ciclo y presencia de crujidos y silibancias con el mismo nombre en fotmato .txt. 

$
 \begin{pmatrix}
  ti & tf &  & c & w\\
  1 & 2 &  & 0 & 1\\ 
  2.2 & 3 &  & 1 & 0\\
  3.5 & 4 &  & 1 & 1\\
  4.1 & 5 &  & 0 & 0\\
 \end{pmatrix}$

In [10]:
IPython.display.Audio(directory+archivos[32]+".wav")

Output hidden; open in https://colab.research.google.com to view.

In [11]:
path=directory + archivos[33] +'.txt'
df_annot = pd.read_csv(path,sep="\t",names=["ti","tf","c","w"])
df_annot["c/w"]=np.array(np.logical_and(df_annot["c"].values,df_annot["w"].values),dtype=np.int8)
df_annot["n"]=np.array(np.logical_not(np.logical_or(df_annot["c"].values,df_annot["w"].values)),dtype=np.int8)
df_annot["duration"]=df_annot["tf"]-df_annot["ti"]
df_annot

,ti,tf,c,w,c/w,n,duration
0,0.022,4.764,1,0,0,0,4.742
1,4.764,9.621,1,0,0,0,4.857
2,9.621,14.921,0,0,0,1,5.300
3,14.921,19.579,0,0,0,1,4.658
4,19.579,19.936,0,0,0,1,0.357


In [12]:
df_annot[["c","w","c/w","n"]].sum().values

array([2, 0, 0, 3])

In [13]:
df_annot[df_annot["duration"]<2][["c","w","c/w","n"]].sum().values

array([0, 0, 0, 1])

In [14]:
def respiratory_cycle_information(file,min_time=1):
  '''
  Esta función extrae los ciclos respiratorios.
  Parametros
    s: nombre del archivo a leer (sin .txt)
  Devuelve 
    df_annot: conjunto de anotaciones
  '''

  path=directory + file +'.txt'
  df_annot = pd.read_csv(path,sep="\t",names=["ti","tf","c","w"])
  df_annot["c/w"]=np.array(np.logical_and(df_annot["c"].values,df_annot["w"].values),dtype=np.int8)
  df_annot["n"]=np.array(np.logical_not(np.logical_or(df_annot["c"].values,df_annot["w"].values)),dtype=np.int8)
  df_annot["duration"]=df_annot["tf"]-df_annot["ti"]

  return df_annot.shape[0],np.concatenate((df_annot[["c","w","c/w","n"]].sum().values,
                         df_annot[df_annot["duration"]>min_time][["c","w","c/w","n"]].sum().values),axis=0)

def progress(value, max=100):
    return HTML("""
        <progress
            value='{value}'
            max='{max}',
            style='width: 80%'
        >
            {value}
        </progress>
    """.format(value=value, max=max))

In [15]:
respiratory_cycle_information(archivos[33],min_time=1)

(5, array([2, 0, 0, 3, 2, 0, 0, 2]))

In [16]:
df1_new=df1
df1_new[["c","w","c/w","n","c2","w2","c/w2","n2"]]=0

out = display(progress(0, 100), display_id=True)
verificar=0
num_archivos=len(archivos)
for i in range(num_archivos):#recorrer la lista de archivos
  
  ver,cycles_info = respiratory_cycle_information(df1["File"][i],min_time=1)
  verificar+=ver
  df1_new.loc[i,["c","w","c/w","n","c2","w2","c/w2","n2"]] = cycles_info
  out.update(progress(i, num_archivos))

In [17]:
verificar

6898

In [ ]:
df1_new

,File,Patient,Equipment,mc/sc,Diagnosis,c,w,c/w,n,c2,w2,c/w2,n2
0,185_1b1_Pl_sc_Litt3200,185,Litt3200,sc,COPD,0,0,0,9,0,0,0,9
1,185_1b1_Lr_sc_Litt3200,185,Litt3200,sc,COPD,0,0,0,7,0,0,0,7
2,185_1b1_Pr_sc_Litt3200,185,Litt3200,sc,COPD,0,0,0,11,0,0,0,11
3,186_2b2_Lr_mc_AKGC417L,186,AKGC417L,mc,COPD,5,0,0,0,5,0,0,0
4,186_2b2_Al_mc_AKGC417L,186,AKGC417L,mc,COPD,0,1,0,4,0,1,0,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
915,183_1b1_Tc_sc_Meditron,183,Meditron,sc,Healthy,0,0,0,4,0,0,0,3
916,184_1b1_Ar_sc_Meditron,184,Meditron,sc,Healthy,0,0,0,8,0,0,0,8
917,185_1b1_Al_sc_Litt3200,185,Litt3200,sc,COPD,0,0,0,7,0,0,0,7
918,185_1b1_Ar_sc_Litt3200,185,Litt3200,sc,COPD,0,0,0,11,0,0,0,11


In [ ]:
df1_new.to_csv('/content/drive/MyDrive/Respiratory Sounds Final/Resultados/ExploracionEtiquetas.csv')

In [ ]:
df=pd.read_csv('/content/drive/MyDrive/Respiratory Sounds Final/Resultados/ExploracionEtiquetas.csv')
df[["c2","w2","c/w2","n2"]].sum()

c2      2332
w2      1361
c/w2     503
n2      3406
dtype: int64

In [ ]:
sum(df['n'])+sum(df['c/w'])+1864+886

6898

In [38]:
import pandas as pd
from pandas import ExcelWriter


df1 = pd.DataFrame({"AAA": (1, 2, 3),
                    "BBB": (5, 7, 11)})

df2 = pd.DataFrame({"CCC": (13,17, 19),
                    "DDD": (23,29, 31)})

with ExcelWriter('prueba.xlsx') as writer:
    df1.to_excel(writer, 'INTRO')
    df2.to_excel(writer, 'DF_READ')

In [41]:
pd.read_excel('prueba.xlsx','DF_READ')

,Unnamed: 0,CCC,DDD
0,0,13,23
1,1,17,29
2,2,19,31
